# NLP Research: Классификация и Суммаризация отзывов


## Цель исследования
В этом модуле мы проводим эксперименты с методами обработки естественного языка (NLP) для анализа отзывов Amazon. 
Наша задача - разработать конвейер, который может:
1.  Определять тональность текста (Sentiment Analysis).
2.  Классифицировать тип обращения (Жалоба/Вопрос/Благодарность).
3.  Генерировать краткое содержание (Summarization) длинных отзывов.

## Методология
* **EDA:** Анализ распределения длин текстов и баланса классов.
* **Heuristics:** Создание Baseline-классификатора на основе ключевых слов.
* **Fine-tuning:** Дообучение `DistilBERT` для классификации тональности.
* **Seq2Seq:** Использование `Flan-T5` для суммаризации текста с применением стратегии чанкинга (chunking).

# Load dataset

Здесь все просто, стандартная загрузка.

In [ ]:
# Load dataset
from datasets import load_dataset

# Загружаем классический датасет Amazon Polarity
ds = load_dataset('amazon_polarity')

In [ ]:
print(ds)

In [ ]:

# датасет содержит поля 'title', 'content' и 'label' (0 - neg, 1 - pos)(то что надо для классификации)
print("примеры:")
for i in range(5):
    print('---')
    print(ds['train'][i])
    print(f'label: {ds["train"][i]["label"]}')

# Разведочный анализ данных
---
## 1. Exploratory Data Analysis (EDA)
Для удобства анализа преобразуем часть датасета в **Pandas DataFrame**. 
Нам важно понять распределение длины текстов, так как это влияет на выбор `max_length` при токенизации и стратегии суммаризации.

In [ ]:
# создание небольшого датасета для теста 
small_train = ds['train'].select(range(2000))
small_test = ds['test'].select(range(2000))

In [ ]:
import pandas as pd
import numpy as np

def ds_to_df(dataset, text='content', label_col='label', n = 2000):
    """Преобразует датасет HuggingFace в DataFrame pandas."""
    items = dataset.select(range(min(len(dataset), n)))
    df = pd.DataFrame({'text':[x[text] for x in items], 'label':[x[label_col] for x in items]})
    return df

df = ds_to_df(small_train, n=2000)
# print(df['label'].value_counts())
# df['char_len'] = df['text'].apply(len)

# print(f"\nChar_len stats: {df['char_len'].describe()}")

# print("\nLong text:\n", df.loc[df['char_len'].idxmax(), 'text'][:500])
# print("\nShort text:\n", df.loc[df['char_len'].idxmin(), 'text'])

# 1. Проверка баланса классов
print("Распределение меток (0=Neg, 1=Pos):")
print(df['label'].value_counts())
count_neg , count_pos = df['label'].value_counts().sort_index().values
print(f"Распределение: Neg = {count_neg} ({count_neg/len(df)*100:.2f}%), Pos = {count_pos} ({count_pos/len(df)*100:.2f}%)")

# 2. Анализ длины текстов (в символах)
df['char_len'] = df['text'].apply(len)
print(f"\nСтатистика длины текстов:\n{df['char_len'].describe()}")

# Посмотрим на выбросы (самый длинный и самый короткий)
print("\n--> Longest text sample (first 500 chars):\n", df.loc[df['char_len'].idxmax(), 'text'][:500])
print("\n--> Shortest text sample:\n", df.loc[df['char_len'].idxmin(), 'text'])

# Эвристическая классификация (Rule-based)
---
## 2. Rule-Based Classification (Baseline)
Помимо тональности, нам важно понимать *интент* пользователя (жалоба, вопрос, похвала). 
В качестве базового решения (Baseline) реализуем классификатор на основе ключевых слов. В продакшене это может служить быстрым пре-фильтром.

In [ ]:
# --- Словари ключевых слов ---
# жалоба 
COMPLAINT_KW = ['not', 'no', 'dirty', 'broken', 'complain', 'problem', 'issue', 'refund', 'late', 'rude', 'terrible']
# комплименты
PRAISE_KW = ['great', 'excellent', 'clean', 'friendly', 'love', 'amazing', 'perfect', 'good', 'recommend']
# вопрос
QUESTION_KW = ['how', 'what', 'where', 'when', 'why', 'is there', 'do you', 'could you']
# предложение 
SUGGESTION_KW = ['should', 'could', 'would be', 'suggest', 'recommendation', 'idea']

def classify_type_rule(text: str) -> str:
    """
    Простая эвристика: проверяет наличие ключевых слов в тексте.
    Приоритет: Жалоба -> Похвала -> Вопрос -> Предложение
    """
    t = text.lower()
    for kw in COMPLAINT_KW:
        if kw in t:
            return 'complaint'
    for kw in PRAISE_KW:
        if kw in t:
            return 'praise'
    for kw in QUESTION_KW:
        if kw in t:
            return 'question'
    for kw in SUGGESTION_KW:
        if kw in t:
            return 'suggestion'
    return 'other'

# Тестируем на случайных примерах
print("--- Тест эвристики ---")
sample_text = df['text'].sample(9, random_state=42).tolist()
for s in sample_text:
    print('-----\n', s[:100])
    print('----> type:', classify_type_rule(s))
    


# Подготовка данных для обучения (Tokenization)
---
## 3. Подготовка данных для Fine-Tuning
Мы будем дообучать модель `DistilBERT` для классификации тональности. 
На этом этапе мы:
1. Создаем маппинг меток (хотя для binary classification это опционально, здесь мы добавляем `type_id` для демонстрации возможности мультиклассового обучения).
2. Токенизируем тексты (превращаем в `input_ids`).


In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset
import pandas as pd

MODEL_NAME = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# создаем DataFrame c колонами: text, sentiment_label(0/1), type_label(0,...,4)
df_small = pd.DataFrame({
    'text': [x['content'] for x in small_train],
    'labels': [x['label'] for x in small_train]
}) 

# добавляем колонку с типом отзыва по эвристике
df_small['type'] = df_small['text'].apply(classify_type_rule)

type2id = {t:i for i, t in enumerate(df_small['type'].unique())}
id2type = {i:t for t,i in type2id.items()}
df_small['type_id'] = df_small['type'].map(type2id)

# Создаем маппинг категорий в ID
print(f'type2id: {type2id}')
print(df_small.head(5))

# Преобразуем в датасет HuggingFace
hf_ds = Dataset.from_pandas((df_small[['text', 'labels', 'type_id']]))
print(f'shape hf_df:{hf_ds.shape}')

In [ ]:
# --- Токенизация ---

def tokenizer_fn(batch):
    # Truncation=True обрезает тексты длиннее 256 токенов (для скорости обучения)
    return tokenizer(batch['text'], truncation=True, padding='max_length', max_length=256)
print("Токенизация данных...")
hf_ds = hf_ds.map(tokenizer_fn, batched=True)
# hf_ds = hf_dds_to_dfs.rename_column('sentiment', 'labels_sentiment')
# hf_ds = hf_ds.rename_column('type_id', 'labels_type')
# hf_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'labels_type'])

# Удаляем текстовые колонки, оставляем только тензоры для PyTorch
hf_ds = hf_ds.remove_columns([c for c in hf_ds.column_names if c not in ('input_ids','attention_mask','labels')])
hf_ds.set_format(type='torch', columns=['input_ids','attention_mask','labels'])
print(f"Формат данных для модели: {hf_ds}")

# Обучение модели (Fine-tuning)
---
## 4. Model Fine-Tuning (DistilBERT)
Используем `Trainer API` от Hugging Face для дообучения модели. 
* **Модель:** DistilBERT (легкая и быстрая).
* **Задача:** Binary Classification (Sentiment).
* **Метрика:** Accuracy.


In [ ]:
# Проверка окружения
import torch
import transformers
import accelerate 
print(torch.__version__)
print(accelerate.__version__)
print(transformers.__version__)

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import evaluate
import numpy as np

# Инициализация модели (num_labels=2 для Pos/Neg)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# Разбиение на train/eval (90/10)
full_split = hf_ds.train_test_split(test_size=0.1, seed=42)
# Ограничиваем размер для демонстрации
train_ds = full_split['train'].select(range(min(4000, len(full_split['train']))))
eval_ds = full_split['test'].select(range(min(800, len(full_split['test']))))

# Метрика
accuracy = evaluate.load('accuracy')
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return accuracy.compute(predictions=preds, references=p.label_ids)

# Параметры обучения
training_args = TrainingArguments(
    output_dir = './models/sentiment-distilbert',
    eval_strategy='epoch',
    save_strategy='epoch',
    per_device_train_batch_size=4, 
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    
)

trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics

)
print("Начало обучения модели...")
trainer.train()



# Суммаризация (Chunking strategy)
---
## 5. Text Summarization (Handling Long Context)
Для суммаризации длинных отзывов мы используем модель `Google Flan-T5`.
Т.к. отзывы могут превышать контекстное окно модели, реализуем стратегию **Chunking**:
1. Разбиваем текст на предложения.
2. Группируем предложения в чанки (блоки) фиксированного размера.
3. Суммаризируем каждый чанк отдельно.

In [ ]:
# summarization 

from transformers import pipeline 
from nltk.tokenize import sent_tokenize
nltk.download('punkt_tab')

# Загрузка NLTK данных для разбиения на предложения
SUM_MODEL = 'google/flan-t5-small'
summarization = pipeline('summarization', model=SUM_MODEL)

def chunk_sentences(text, max_chars=800):
    """
    Разбивает текст на куски (chunks), не разрывая предложения.
    """
    sents = sent_tokenize(text)
    chunks = []
    curr = []
    curr_len = 0
    for s in sents:
        if curr_len + len(s) > max_chars and curr:
            chunks.append(' '.join(curr))
            curr = [s]
            curr_len = len(s)
        else:
            curr.append(s)
            curr_len += len(s)

    if curr:
        chunks.append(' '.join(curr))
    return chunks

# --- Демонстрация работы ---
print("--- Test Chunking Summary ---")
# Берем длинный текст из датасета
sample_long = df_small['text'].iloc[0]
chunks = chunk_sentences(sample_long, max_chars=600)
print(f'Original Text Length: {len(sample_long)} chars')
print(f'Total chunks created: {len(chunks)}')

for c in chunks:
    print(f'--- chunk ---\n{c[:300]}')
    # max_length - длина саммари, min_length - минимальная длина
    out = summarization(c, max_length=50, min_length=10, truncation=True)
    print(f"-> summary: {out[0]['summary_text']}")

# Финальный пайплайн (Inference)
## 6. Unified NLP Pipeline
Собираем все компоненты в единую функцию анализа.
> **Note:** Для демонстрации инференса мы используем готовую fine-tuned модель `sst-2` (чтобы не зависеть от локального обучения выше), но в реальном кейсе здесь подгружалась бы модель из `./models/sentiment-distilbert`.


In [ ]:
def analyze_text(text, do_summary=True):
    """
    Комплексный анализ текста:
    1. Sentiment Analysis (Positive/Negative)
    2. Category Classification (Rule-based)
    3. Summarization (если текст длинный)
    """
    # загрузка пайплайна для сентимент анализа
    from transformers import pipeline as hf_pipeline
    # Используем sst-2
    sent = hf_pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')
    # Обрезаем до 512, так как BERT имеет лимит
    sentiment_result = sent(text[:512])[0]['label'].lower()
    sentiment_label = 'positive' if 'pos' in sentiment_result else 'negative'

    # 2. Категория (эвристика)
    t = classify_type_rule(text)

    # 3. Суммаризация
    summary = None
    if do_summary:
        chunks = chunk_sentences(text, max_chars=600)
        parts = []
        for c in chunks:
            parts.append(summarization(c, max_length=60, min_length=10, truncation=True)[0]['summary_text'])
        # joiu
        summary = ' '.join(parts)
    return {'->sentiment':sentiment_label, 
            '->category': t, 
            '->summary': summary}

# print(analyze_text(df_small['text'].iloc[0], do_summary=True))
# Тестируем на примере
print("--- Final Analysis Result ---")
result = analyze_text(df_small['text'].iloc[0], do_summary=True)
for k, v in result.items():
    print(f"{k}: {v}")